# Plan & Execute Agent Tutorial

This notebook demonstrates the **Plan & Execute** pattern for complex, multi-step tasks.

```
Complex Task → PLAN (create subtasks) → EXECUTE (run each) → SYNTHESIZE (combine results)
```

### When to Use Plan & Execute vs ReAct

| Use Case | ReAct | Plan & Execute |
|----------|-------|----------------|
| Quick lookups | ✅ | |
| Multi-step research | | ✅ |
| Unknown number of steps | ✅ | |
| Clear subtask dependencies | | ✅ |

## Setup

In [ ]:
# Install miiflow-llm
!pip install -q miiflow-llm nest_asyncio

import nest_asyncio
nest_asyncio.apply()

In [ ]:
import os
from getpass import getpass

if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass("Enter your OpenAI API key: ")

print("API key configured!")

## Define Tools

In [ ]:
from miiflow_llm.core.tools import tool


@tool("search_web", "Search the web for information")
def search_web(query: str) -> str:
    """Search for information on the web."""
    return f"""
    Search results for '{query}':
    1. Wikipedia article about {query}
    2. Recent news: {query} trends
    3. Expert analysis of {query}
    """


@tool("analyze_data", "Analyze data and provide insights")
def analyze_data(data: str, analysis_type: str = "summary") -> str:
    """Analyze provided data."""
    return f"""
    Analysis ({analysis_type}):
    - Data received: {data[:100]}...
    - Key findings: 3 main patterns identified
    - Confidence: High
    """


@tool("generate_report", "Generate a formatted report")
def generate_report(title: str, sections: str) -> str:
    """Generate a report from sections."""
    return f"""
    # {title}

    ## Executive Summary
    This report covers: {sections}

    ## Key Findings
    - Finding 1: Significant trends identified
    - Finding 2: Areas for improvement noted

    ## Conclusion
    Based on analysis, actionable steps are recommended.
    """


print("Tools defined: search_web, analyze_data, generate_report")

## Example 1: Basic Plan & Execute

The agent will:
1. Create a plan with subtasks
2. Execute each subtask
3. Combine results into a final answer

In [ ]:
from miiflow_llm import LLMClient, Agent, AgentType

# Create client
client = LLMClient.create("openai", model="gpt-4o-mini")

# Create Plan & Execute agent
agent = Agent(
    client,
    agent_type=AgentType.PLAN_AND_EXECUTE,
    max_iterations=10,
    system_prompt="You are an analyst. Create structured plans for research tasks.",
)

agent.add_tool(search_web)
agent.add_tool(analyze_data)
agent.add_tool(generate_report)

print("Plan & Execute agent created!")

In [ ]:
# A task that benefits from planning
task = """
Research the latest trends in artificial intelligence,
analyze the key developments, and create a brief report.
"""

print("Task:", task)
print("\nExecuting with Plan & Execute...\n")

result = await agent.run(task)
print(result.data)

## Example 2: Watch the Planning Process (Streaming)

Stream the agent's execution to see:
1. The planning phase (creating subtasks)
2. Each subtask being executed
3. The final synthesis

In [ ]:
from miiflow_llm import RunContext
from miiflow_llm.core.react import PlanExecuteEventType

task = "Find information about Python web frameworks and compare them"

print("Task:", task)
print("=" * 50)

context = RunContext(deps=None, messages=[])

async for event in agent.stream(task, context):
    event_type = event.event_type
    
    if event_type == PlanExecuteEventType.PLANNING_START:
        print("\n[PHASE 1: PLANNING]")
        print("Creating execution plan...\n")
    
    elif event_type == PlanExecuteEventType.PLANNING_COMPLETE:
        subtask_count = event.data.get("subtask_count", 0)
        print(f"\nPlan created with {subtask_count} subtasks!")
        print("\n[PHASE 2: EXECUTION]")
    
    elif event_type == PlanExecuteEventType.SUBTASK_START:
        desc = event.data.get("description", "")[:60]
        print(f"\n  Subtask: {desc}...")
    
    elif event_type == PlanExecuteEventType.SUBTASK_COMPLETE:
        result_preview = str(event.data.get("result", ""))[:80]
        print(f"  Done: {result_preview}...")
    
    elif event_type == PlanExecuteEventType.FINAL_ANSWER:
        print("\n" + "=" * 50)
        print("[PHASE 3: FINAL ANSWER]")
        print("=" * 50)
        print(event.data.get("answer", ""))

## Key Takeaways

### Plan & Execute Pattern
1. **PLAN**: Break down complex tasks into subtasks with dependencies
2. **EXECUTE**: Run subtasks in order, respecting dependencies
3. **SYNTHESIZE**: Combine results into comprehensive answer

### When to Use
- Multi-step research tasks
- Tasks with clear phases (gather → analyze → report)
- Complex workflows where organization helps

### Key Parameters
- `agent_type=AgentType.PLAN_AND_EXECUTE` - enables planning mode
- `max_iterations=10+` - complex tasks need more iterations

### vs ReAct
- **ReAct**: Flexible, adaptive, lower overhead → simple queries
- **Plan & Execute**: Structured, organized, visible planning → complex research